## Import das bibliotecas e do Data Set do Artigo
Nesta seção, importamos as bibliotecas necessárias e montamos o Google Drive para acessar o dataset.

In [1]:
!pip install scapy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.7 MB/s eta 0:00:0000:01


In [2]:
from google.colab import drive
from scapy.all import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Conectando ao Google Drive para acessar os arquivos do dataset.

In [3]:
file_path = "../input/auteth-intrusion-dataset-treated"

## Processamento dos dados do Data Set
Nesta seção, definimos constantes e funções para processar os dados do dataset.

### Constantes
Definimos algumas variáveis constantes que serão utilizadas no processamento dos dados.

Para o processamento dos dados, vamos estabalecer algumas variáveis constantes:

In [4]:
avtp_pkt_sz = 438                      #tamanho do pacote AVTP
window_sz = 44                         #tamanho de janela ideal indicado pelo artigo
num_bytes_observados = 58              #apenas os 58 primeiros bytes do pacote serao observados
num_colunas = num_bytes_observados * 2 #segundo o artigo, uma feature eh representada por meio byte (ou um nibble)

### Funções utilizadas para a leitura
Definimos funções para transformar pacotes em arrays numpy, selecionar bytes relevantes, calcular variações e gerar features.

In [5]:
import numpy as np

# Transforma os pacotes em um array numpy
def pcap_to_npArr(raw_pkts):
  packets_list = [np.frombuffer(raw(packet), dtype='uint8') for packet in raw_pkts if len(raw(packet))==438]

  # Filtra pacotes com o tamanho correto (438 bytes)
  pkts_array = np.array(packets_list, dtype=object)

  return pkts_array

In [6]:
# Captura os primeiros 58 bytes dos pacotes
def select_bytes(packets, start_byte=0, end_byte=58):
   return np.array(packets[:,start_byte:end_byte], dtype='uint8')

# Calcula as variações dos bytes dos pacotes
def get_variations_mod(packets):
  variations = np.diff(packets, axis=0).astype(np.int16)
  variations_mod = np.mod(variations, 256)
  return np.array(variations_mod, dtype='uint8')

# Divide um byte em dois nibbles
def split_nibbles(byte):
    low_nibble = byte & 0xf
    high_nibble = (byte>>4) & 0xf
    return [high_nibble, low_nibble]

In [7]:
# Gera features a partir dos pacotes
def generate_features(packets):
   relevant_bytes = select_bytes(packets)
   byte_variations = get_variations_mod(relevant_bytes)
   features = []
   for variation_row in byte_variations:
      feature_row = []
      for byte_variation in variation_row:
        feature_row.extend(split_nibbles(byte_variation))
      features.append(feature_row)
   return np.array(features, dtype = 'uint8')

In [8]:
# Verifica as janelas de pacotes e seus rótulos
def check_windows(pkts, labels):
  ansX, anxY = [], []

  for i in range(0, len(pkts) - window_sz + 1, 1):
    ansX.append(pkts[i:i+window_sz])
    anxY.append(labels[i+window_sz-1])

  return np.array(ansX), np.array(anxY)


### Tratamento da base de dados
Nesta seção, carregamos os pacotes do dataset, geramos as features e rotulamos os dados.

In [9]:
# Carrega os pacotes de replay
replay_frames = f'{file_path}/single-MPEG-frame.pcap'
pkt_list_replay_raw = rdpcap(replay_frames)
pkt_list_replay = pcap_to_npArr(pkt_list_replay_raw)
print(pkt_list_replay)

[[145 224 240 ... 64 156 185]
 [145 224 240 ... 196 64 43]
 [145 224 240 ... 54 80 127]
 ...
 [145 224 240 ... 247 227 163]
 [145 224 240 ... 38 137 196]
 [145 224 240 ... 226 207 242]]


In [10]:
# Define os caminhos dos arquivos de pacotes benignos e maliciosos
#benign_indoors_paths = [f'{file_path}/indoors_01_original.pcap', f'{file_path}/indoors_02_original.pcap']
#benign_outdoor_paths = [f'{file_path}/driving_01_original.pcap', f'{file_path}/driving_02_original.pcap']
#malicious_indoors_paths = [f'{file_path}/indoors_01_injected.pcap', f'{file_path}/indoors_02_injected.pcap']
malicious_outdoor_paths = [f'{file_path}/driving_01_injected.pcap', f'{file_path}/driving_02_injected.pcap']

In [11]:
# Inicializa arrays para armazenar os dados e rótulos
x = np.empty(shape=(0, window_sz, num_colunas), dtype='uint8')
y = np.array([], dtype='uint8')

# Processa os pacotes maliciosos internos
for path in malicious_outdoor_paths:
  pkt_list = rdpcap(path)
  pkt_list_arr = pcap_to_npArr(pkt_list)
  pp_pkt_list_arr = generate_features(pkt_list_arr)

  labels_list = []

  # Rotula os pacotes
  for pkt in pkt_list_arr:
    if(np.any(np.all(pkt == pkt_list_replay, axis=1))):
      labels_list.append(1)
    else:
      labels_list.append(0)

  labels_arr = np.array(labels_list, dtype='uint8')

  # Verifica as janelas de pacotes e seus rótulos
  x_aux, y_aux = check_windows(pp_pkt_list_arr, labels_arr)
  x = np.append(x, x_aux, axis=0)
  y = np.append(y, y_aux, axis=0)


In [12]:
# Verifica se o dataset foi rotulado corretamente

outdoors_unique, outdoors_counts = np.unique(np.array(y), return_counts=True)

# Dindoors tem 1,494,257 pacotes benignos e 376,236 pacotes injetados [Informação do artigo]
print(f"Outdoors has {outdoors_counts[0]} bening Xis and {outdoors_counts[1]} injected Xis")

Outdoors has 1494255 bening Xis and 376236 injected Xis


In [14]:
# Salva os dados processados em arquivos comprimidos
np.savez_compressed("/kaggle/working/x_outdoor_packets_agg.npz", x)
np.savez_compressed("/kaggle/working/y_outdoor_label_agg.npz", y)